In [2]:
# Import all packages and library

# Import package to scan hyperparameter
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix

# Import package to reprocess the data
import numpy as np
import pandas as pd
import random

# Import properties from keras
from keras import models
from keras.layers import Dense, Dropout, Activation
from keras import regularizers
from keras import initializers

# Import keras items
from keras.optimizers import Adam, Adadelta, SGD
from keras.activations import relu, sigmoid
from keras.layers.advanced_activations import PReLU
from keras.losses import binary_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [3]:
# Get all of the data and reprocess them

# Get the reprocessed data from .npy file
x_train = np.load('../r-scripts/getting-data-current/data-sets/x_train.npy')
y_train = np.load('../r-scripts/getting-data-current/data-sets/y_train.npy')

x_dev = np.load('../r-scripts/getting-data-current/data-sets/x_val.npy')
y_dev = np.load('../r-scripts/getting-data-current/data-sets/y_val.npy')

x_test = np.load('../r-scripts/getting-data-current/data-sets/x_test.npy')
y_test = np.load('../r-scripts/getting-data-current/data-sets/y_test.npy')

# This Section is used to shuffle the data

# This Section is used to shuffle the data

# Aggregates elements
data_training = list(zip(x_train, y_train))
data_development = list(zip(x_dev, y_dev))
data_testing = list(zip(x_test, y_test))

# Shuffle the aggragated element on the list
random.shuffle(data_training)
random.shuffle(data_development)
random.shuffle(data_testing)

# Combine data training dan data development become one list of data train

data_train = data_training + data_development

# Split the shuffled data
x_train, y_train = zip(*data_train)
x_test, y_test = zip(*data_testing)

# Unpack the tuples
x_train = np.array(list(x_train))
y_train = np.array(list(y_train))
x_test = np.array(list(x_test))
y_test = np.array(list(y_test))

# Reshape the datasets
x_train = x_train.reshape(615, 4034 * 20)
x_test = x_test.reshape(150, 4034 * 20)

In [4]:
# Define the model and function

# Define the model base
def build_fc_model(# Hyperparameters as parts of model designs (building blocks)
	               input_num_hidden_units = 2,
                   num_hidden_layers = [0],
                   activation_function = 'relu',

                   # Hyperparameters as part of optimization and regularization of the models
                   l2_rate = 0.01,
                   input_dropout_rates = 0.5,
                   dropout_rates = 0.5,
                   optim_methods = 'Adam',
                   batch_norm = "yes"
                   ):

	# Add the input layer
    model = models.Sequential()
    model.add(Dense(input_num_hidden_units,
              activation = 'relu',
              kernel_regularizer = regularizers.l2(l2_rate),
              kernel_initializer=initializers.glorot_uniform(seed=3),
              input_dim = x_train.shape[1]))
    model.add(Dropout(input_dropout_rates))

    # Add the hidden layers
    for num in range(len(num_hidden_layers)):
        if num_hidden_layers[num] == 0:
            continue
        else:
            model.add(Dense(num_hidden_layers[num]))

            # Add batch normaization before adding the activation layers
            if batch_norm == "yes":
            	model.add(BatchNormalization())
            else:
            	continue

            model.add(Activation(activation_function))
            model.add(Dropout(dropout_rates))

    # Add the output layer
    model.add(Dense(1,
              activation = 'sigmoid'))

    # Compile the model defined
    model.compile(optimizer = optim_methods,
                  loss = 'binary_crossentropy',
                  metrics = ['acc'])

    # Print the summary of the model
    print(model.summary())

    return model

In [13]:
# Predict the results of hyperparamaters tuning

# Define function to fit the model
def train_fc_model(batch_sizes = None, num_epochs = None):
      model.fit(x = x_train,
                y = y_train,
                batch_size = batch_sizes,
                epochs = num_epochs,
                verbose = 0,
                shuffle = 1)

# Define the function to calculate sensitivity and specificity
def sensitivity_specificity(predictions, y_test, mode='binary'):
    if mode == 'binary':
        # Determine positive class predictions
        index = predictions > 0.5
        predictions = np.zeros(predictions.shape)
        predictions[index] = 1
        # No need to modify y_test since it consists of zeros and ones already
    else:
        y_test = y_test
        predictions = np.argmax(predictions, axis=-1)

    # In the binary classification case as we create, we can extract tn, fp, fn, tp as follows
    tn, fp, fn, tp = confusion_matrix(y_test, predictions, labels = [0, 1]).ravel()

    # Sensitivity = TP / (TP + FN)
    sensitivity = tp / (tp + fn)

    # Specificity = TN / (TN + FP)
    specificity = tn / (tn + fp)

    # Precision = TP / (TP + FP)
    precision = tp / (tp + fp)

    # Return sensitivity, specificity, precision
    return(sensitivity, specificity, precision)


# Define function to evaluate and predict
def evaluate_predict_fc_model():
  loss, acc = model.evaluate(x_test, y_test, verbose = 0)
  prediction = model.predict(x_test)
  sensitivity, specificity, precision = sensitivity_specificity(prediction, y_test, mode='binary')
  return acc, sensitivity, specificity, precision

# make prediction
num_epochs = [1]
num_batch_size = [4, 8, 16]

result_list = []
columns_names = ['Epochs', 
                 'Batch Size', 
                 'Acc']

for i in range(len(num_epochs)):
  for j in range(len(num_batch_size)):
    model = build_fc_model(# Hyperparameters as parts of model designs (building blocks)
	                       input_num_hidden_units = 3,
                           num_hidden_layers = [1, 3],
                           activation_function = 'relu',

                           # Hyperparameters as part of optimization and regularization of the models
                           l2_rate = 0.01,
                           input_dropout_rates = 0.5,
                           dropout_rates = 0.5,
                           optim_methods = 'SGD',
                           batch_norm = 'no')
    train_fc_model(batch_sizes = num_batch_size[j], num_epochs = num_epochs[i])
    acc, sensitivity, specifity, precision = evaluate_predict_fc_model()
    result_line = np.array((num_epochs[i],
                            num_batch_size[j],
                            acc))
    result_list.append(result_line[:])
    result_array = np.asarray(result_list)
    df_results = pd.DataFrame(result_array,
                              columns = columns_names)
    
# df_results.to_csv('df_result_manual_model1.csv')
print(df_results)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_113 (Dense)            (None, 3)                 242043    
_________________________________________________________________
dropout_29 (Dropout)         (None, 3)                 0         
_________________________________________________________________
dense_114 (Dense)            (None, 1)                 4         
_________________________________________________________________
dense_115 (Dense)            (None, 3)                 6         
_________________________________________________________________
dense_116 (Dense)            (None, 1)                 4         
Total params: 242,057
Trainable params: 242,057
Non-trainable params: 0
_________________________________________________________________
None
   Epochs  Batch Size   Acc
0    40.0        16.0  0.74
